<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-14 11:10:22
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  9.00 K
Current:  1.36 C
-------------------
Today PnL: -7.20 K (-0.05%)
Current PnL: -14.07 L (-9.81%)
CY Booked + Current PnL: -7.11 L (-4.96%)
-------------------
Total profit:  4.68 L
Total loss:  -18.75 L
-------------------
Total Booked + Current PnL: 19.87 L (17.05%)
Total Booked PnL: 33.94 L (29.12%)
Curr Year Booked PnL: 6.96 L (5.12%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 72.61 L (53.37%)
Deployed:  1.17 C
Current:  1.36 C
CAGR/XIRR %: 10.31%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.00,-222.47,H-SC,85.42,88296.0,-7020.0,7055.0,-0.82,-7.36,7.99,0.04,157.0,-1.00,0.65,74.15,OX40N,NTT,MISC
87,UJJIVANSFB,52.77,53.00,52.86,M-SC,60.42,132894.0,-9585.0,10206.0,3.53,-6.73,7.68,0.44,247.0,-0.94,0.98,56.85,OX40N,NTT,BANKS
71,SBIN,760.30,863.00,-11.51,M-LC,63.54,215220.0,13740.0,13473.0,0.43,6.82,6.26,13.51,77.0,1.02,1.58,20.28,XY25,NTT,BANKS
39,HINDALCO,651.95,761.55,-22.38,H-LC,68.75,107560.0,3248.0,14284.0,0.19,3.11,13.28,16.81,27.0,0.23,0.79,19.66,X5K,ATH,METALS
52,JPPOWER,18.73,26.20,-9.01,L-SC,100.00,182888.0,41046.0,15527.0,2.20,28.94,8.49,39.88,262.0,2.64,1.35,85.34,XY24,NTT,POWER


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BERGEPAINT,561.33,680.00,-19.78,H-MC,34.38,226962.0,-377.0,48434.0,-2.12,-0.17,21.34,21.14,117.0,-0.01,1.67,27.07,XY24,NTT,PAINTS
60,NATIONALUM,189.63,247.44,-49.40,H-MC,59.38,101188.0,684.0,29952.0,0.75,0.68,29.60,30.49,97.0,0.02,0.74,33.31,MH,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.00,52.86,M-SC,60.42,132894.0,-9585.0,10206.0,3.53,-6.73,7.68,0.44,247.0,-0.94,0.98,56.85,OX40N,NTT,BANKS
28,DIGITIDE,188.38,237.72,NaN,NaN,8.33,49207.0,7952.0,2854.0,3.32,19.27,5.80,26.19,272.0,2.79,0.36,15.85,XY24,ATH,IT
45,INDIAMART,2327.09,4911.36,-45.65,H-SC,86.46,141510.0,18174.0,118798.0,2.51,14.74,83.95,111.05,123.0,0.15,1.04,38.65,AR,ATH,MISC
41,HINDZINC,514.80,730.22,15.78,H-LC,1.04,86930.0,-16030.0,59112.0,2.29,-15.57,68.00,41.85,24.0,-0.27,0.64,14.41,X5K,ATH,METALS
52,JPPOWER,18.73,26.20,-9.01,L-SC,100.00,182888.0,41046.0,15527.0,2.20,28.94,8.49,39.88,262.0,2.64,1.35,85.34,XY24,NTT,POWER


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3176.7,NaN,NaN,95.83,76902.0,42453.0,2515.0,-3.17,123.24,3.27,130.54,274.0,16.88,0.57,17.60,AR,ATH,ELECTRICAL
14,BERGEPAINT,561.33,680.0,-19.78,H-MC,34.38,226962.0,-377.0,48434.0,-2.12,-0.17,21.34,21.14,117.0,-0.01,1.67,27.07,XY24,NTT,PAINTS
19,CAMPUS,294.86,393.0,-24.03,M-SC,37.50,158324.0,-9451.0,65293.0,-1.82,-5.63,41.24,33.28,206.0,-0.14,1.16,22.58,XY24,NTT,FOOTWEAR
65,RELAXO,902.64,1176.0,-30.18,H-SC,98.96,63110.0,-52428.0,87420.0,-1.55,-45.38,138.52,30.28,131.0,-0.60,0.46,23.29,X40N,NTT,FOOTWEAR
69,SAMMAANCAP,170.35,326.0,-164.77,M-SC,29.17,75162.0,-27048.0,120440.0,-1.53,-26.46,160.24,91.37,212.0,-0.22,0.55,22.01,XY25,NTT,FINANCE


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,52.86,M-SC,60.42,132894.0,-9585.0,10206.0,3.53,-6.73,7.68,0.44,247.0,-0.94,0.98,56.85,OX40N,NTT,BANKS
92,VIPIND,488.80,489.0,-222.47,H-SC,85.42,88296.0,-7020.0,7055.0,-0.82,-7.36,7.99,0.04,157.0,-1.00,0.65,74.15,OX40N,NTT,MISC
79,SYMPHONY,1306.42,1306.0,-30.13,M-SC,46.88,148986.0,-22155.0,22095.0,-1.04,-12.95,14.83,-0.03,191.0,-1.00,1.10,7.08,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,93.75,148998.0,-25561.0,25598.0,0.12,-14.64,17.18,0.02,133.0,-1.00,1.10,29.20,OX40N,NTT,PAINTS
81,TATAELXSI,7332.28,7332.0,-17.65,X-MC,19.79,86380.0,-16272.0,16265.0,1.71,-15.85,18.83,-0.00,59.0,-1.00,0.64,31.31,OX40N,NTT,IT


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TCS,3794.03,4998.00,-22.56,X-LC,5.21,252377.0,-43557.0,137470.0,-0.93,-14.72,54.47,31.73,1.0,-0.32,1.86,1.02,X40,BTT,IT
48,INFY,1461.46,2275.00,-12.42,X-LC,22.92,281442.0,19841.0,125776.0,-1.42,7.58,44.69,55.67,2.0,0.16,2.07,14.15,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-10.23,X-LC,90.62,281042.0,10695.0,23945.0,-0.41,3.96,8.52,12.81,5.0,0.45,2.07,17.54,XY25,NTT,FMCG
17,BRITANNIA,4983.33,6446.05,15.95,X-LC,70.83,265972.0,36739.0,30534.0,0.41,16.03,11.48,29.35,10.0,1.20,1.96,26.38,XY25,ATH,FMCG
74,SIEMENS,4406.90,7969.85,-7.62,X-LC,12.50,78050.0,-32122.0,121196.0,0.14,-29.16,155.28,80.85,11.0,-0.27,0.57,14.95,AR,ATH,ELECTRICAL


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3176.7,NaN,NaN,95.83,76902.0,42453.0,2515.0,-3.17,123.24,3.27,130.54,274.0,16.88,0.57,17.60,AR,ATH,ELECTRICAL
52,JPPOWER,18.73,26.2,-9.01,L-SC,100.00,182888.0,41046.0,15527.0,2.20,28.94,8.49,39.88,262.0,2.64,1.35,85.34,XY24,NTT,POWER
59,MEDANTA,1064.65,1486.0,19.36,X-SC,89.58,254528.0,49051.0,32274.0,1.19,23.87,12.68,39.58,65.0,1.52,1.87,36.39,XY24,NTT,HEALTHCARE


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,93.57,M-MC,76.04,231885.0,6923.0,160743.0,0.25,3.08,69.32,74.53,189.0,0.04,1.71,34.63,XY24,BTT,STEEL
54,KPIGREEN,497.21,732.01,29.50,H-SC,81.25,132640.0,7343.0,51822.0,0.98,5.86,39.07,47.22,143.0,0.14,0.98,65.94,MH,ATH,POWER
20,CAMS,3643.00,5250.99,2.54,H-SC,47.92,115083.0,13079.0,31947.0,1.00,12.82,27.76,44.14,126.0,0.41,0.85,31.15,X40N,ATH,MISC
15,BLUESTARCO,1646.70,2337.55,-3.43,H-SC,88.54,181040.0,16370.0,52719.0,0.59,9.94,29.12,41.95,124.0,0.31,1.33,18.15,X40N,ATH,AC
88,UNIONBANK,123.87,163.00,-4.47,M-LC,45.83,165797.0,24957.0,19531.0,1.45,17.72,11.78,31.59,88.0,1.28,1.22,44.02,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3176.70,NaN,NaN,95.83,76902.0,42453.0,2515.0,-3.17,123.24,3.27,130.54,274.0,16.88,0.57,17.60,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-38.18,M-SC,77.08,203904.0,32283.0,129785.0,1.06,18.81,63.65,94.43,201.0,0.25,1.50,50.49,XR,NTT,DURABLES
45,INDIAMART,2327.09,4911.36,-45.65,H-SC,86.46,141510.0,18174.0,118798.0,2.51,14.74,83.95,111.05,123.0,0.15,1.04,38.65,AR,ATH,MISC
54,KPIGREEN,497.21,732.01,29.50,H-SC,81.25,132640.0,7343.0,51822.0,0.98,5.86,39.07,47.22,143.0,0.14,0.98,65.94,MH,ATH,POWER
95,WIPRO,243.46,420.00,-7.85,M-LC,21.88,157821.0,6876.0,102584.0,-1.45,4.56,65.00,72.51,71.0,0.07,1.16,8.71,XR,NTT,IT


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,86.36,NaN,NaN,96.88,17969.0,-31624.0,943.0,-0.53,-63.77,5.25,-61.86,270.0,-33.54,0.13,20.50,XY24,ATH,MISC
24,COFFEEDAY,59.14,80.00,-55.73,L-SC,30.21,66048.0,-47501.0,87553.0,-1.12,-41.83,132.56,35.27,265.0,-0.54,0.49,61.65,XR,NTT,HOTELS
32,ENRIN,1377.95,3176.70,NaN,NaN,95.83,76902.0,42453.0,2515.0,-3.17,123.24,3.27,130.54,274.0,16.88,0.57,17.60,AR,ATH,ELECTRICAL
6,ASIANTILES,75.41,137.00,7205.56,M-SC,31.25,80163.0,-13647.0,90264.0,-0.06,-14.55,112.60,81.67,245.0,-0.15,0.59,55.24,XR,NTT,CERAMICS
92,VIPIND,488.80,489.00,-222.47,H-SC,85.42,88296.0,-7020.0,7055.0,-0.82,-7.36,7.99,0.04,157.0,-1.00,0.65,74.15,OX40N,NTT,MISC


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-9.01,L-SC,100.00,182888.0,41046.0,15527.0,2.20,28.94,8.49,39.88,262.0,2.64,1.35,85.34,XY24,NTT,POWER
65,RELAXO,902.64,1176.00,-30.18,H-SC,98.96,63110.0,-52428.0,87420.0,-1.55,-45.38,138.52,30.28,131.0,-0.60,0.46,23.29,X40N,NTT,FOOTWEAR
3,ALKYLAMINE,2347.98,4546.37,4.24,X-SC,97.92,96965.0,-3998.0,98526.0,0.93,-3.96,101.61,93.63,67.0,-0.04,0.71,44.43,SR,ATH,CHEMICALS
16,BLUSPRING,226.45,86.36,NaN,NaN,96.88,17969.0,-31624.0,943.0,-0.53,-63.77,5.25,-61.86,270.0,-33.54,0.13,20.50,XY24,ATH,MISC
32,ENRIN,1377.95,3176.70,NaN,NaN,95.83,76902.0,42453.0,2515.0,-3.17,123.24,3.27,130.54,274.0,16.88,0.57,17.60,AR,ATH,ELECTRICAL


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.96
1,25,41.88
2,50,70.66


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.85
LC    33.46
MC    21.66
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.17
X40      12.75
XY25     12.13
X40N     11.88
XR        9.97
OX40N     8.25
AR        8.18
MH        1.72
X5K       1.43
X200      1.36
SR        1.19
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.59
M-SC    15.94
H-LC    13.90
X-LC    12.75
H-MC     9.58
X-MC     7.64
M-LC     5.77
X-SC     4.57
M-MC     4.08
L-SC     2.75
L-LC     1.04
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.30,-0.66,34.31
IT,11.83,-13.04,68.59
FINANCE,8.46,-13.39,54.54
BANKS,7.73,-8.48,52.74
MISC,5.92,-19.34,75.32
PAINTS,5.61,-14.71,37.32
ELECTRICAL,5.44,0.25,47.02
HEALTHCARE,4.78,0.73,28.27
AUTO,4.26,-16.90,68.30


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2652763.0,26
XR,1102660.0,13
AR,1040920.0,10
X40,722677.0,10
X40N,549412.0,11
XY25,469201.0,8
OX40N,310374.0,11
SR,182245.0,2
MH,81774.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1988042.0,20
M-SC,1776236.0,21
H-LC,650424.0,12
X-LC,576765.0,8
H-MC,532732.0,8
M-MC,416490.0,3
X-MC,416361.0,6
X-SC,295554.0,5
M-LC,275710.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      898882.0      6
M-SC       XY24      809926.0      7
H-SC       AR        520295.0      3
M-SC       XR        335820.0      4
H-MC       XY24      304670.0      3
M-MC       XY24      301921.0      2
X-LC       X40       300432.0      3
H-SC       XR        221390.0      3
           X40N      208339.0      4
H-LC       X40N      205816.0      4
M-SC       AR        189261.0      2
           OX40N     165533.0      5
H-LC       X40       164006.0      3
M-LC       XY24      159653.0      2
X-MC       X40       158435.0      2
L-SC       XR        151594.0      2
X-SC       XR        123492.0      1
X-LC       AR        121196.0      1
M-SC       XY25      120440.0      1
M-MC       XR        114569.0      1
X-MC       XY25      107979.0      1
M-LC       XR        102584.0      1
X-SC       SR         98526.0      1
X-LC       XY25       97495.0      3
H-SC       OX40N      87314.0      3
H-LC       AR         86191.0      1
M-SC       SR         83719.0      1
H-MC       XY25       82029.0      1
X-MC       XY24       80324.0      1
H-LC       X200       75226.0      1
           X5K        73396.0      2
M-SC       X40        71537.0      1
H-MC       AR         63557.0      1
L-SC       AR         57905.0      1
X-LC       X40N       57642.0      1
X-MC       X40N       53358.0      1
L-MC       XR         53211.0      1
H-SC       MH         51822.0      1
L-LC       XY25       47785.0      1
H-LC       XY24       45789.0      1
X-SC       OX40N      41262.0      2
           XY24       32274.0      1
H-MC       MH         29952.0      1
           X40        28267.0      1
           X40N       24257.0      1
X-MC       OX40N      16265.0      1
L-SC       XY24       15527.0      1
M-LC       XY25       13473.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
